<a href="https://colab.research.google.com/github/abhijeet3922/vision-RAG/blob/main/5_vectordb_as_retriever_colpali_as_reranker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **FAISS vector DB as Retriever (1st pass) and ColPali as Re-ranker (2nd pass)**

This notebook goes one step deeper into treating faster vector DB cosine scoring as 1st pass and then employing ColPali as re-ranker to improve accuracy (SOTA late interaction mechanism).

- The example in the notebook shows where FAISS vector DB retrieval fails to get correct page on the top, ColPali comes to rescue as a re-ranker to fetch the correct page.

- This is certainly very helpful in scaling the solution.


Following are the code components for this notebook:
1. [Installing Libraries & Imports](#)
2. [Loading Visual Language Model (VLM): ColPali](#)
3. [Ingest PDF Data as Embeddings](#)
4. [Setup FAISS vector DB Index](#)
5. [Add Embeddings & Metadata Fields to Index](#)
6. [Retrieve best Matching Page from Vector DB](#)
7. [Rerank Vector DB output using Colpali as Re-ranker](#)
8. [Load Multi-modal LLM: Qwen2.5-VL 3B](#)
9. [Prepare Prompt with Retrieved Page](#)
10. [Generate Answer: Model Inference](#)



### 1. Install Libraries & Imports

In [ ]:
!pip install pdf2image
!pip install colpali-engine==0.3.9
!sudo apt-get install poppler-utils

!pip install faiss-cpu
!pip install langchain_community

In [ ]:
import torch
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from pdf2image import convert_from_path
from torch.utils.data import DataLoader
from colpali_engine.models import ColPali, ColPaliProcessor

### 2. Loading Visual Language Model (VLM): ColPali

In [ ]:
model_name = "vidore/colpali-v1.3"

model = ColPali.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",  # or "mps" if on Apple Silicon
).eval()

processor = ColPaliProcessor.from_pretrained(model_name)

### 3. Ingest PDF Data as Embeddings

In [ ]:
images = convert_from_path('/content/google-alphabet-2024.pdf')
print("Number of pages:", len(images))

In [ ]:
dataloader = DataLoader(images,
                        batch_size=1,
                        shuffle=False,
                        collate_fn=lambda x: processor.process_images(x).to(model.device))

In [ ]:
dataset = []
for batch in tqdm(dataloader):
  with torch.no_grad():
    batch = {k: v.to(model.device) for k,v in batch.items()}
    embeddings = model(**batch)
  dataset.extend(list(torch.unbind(embeddings.to("cpu").to(torch.float32))))

In [ ]:
query = ["What is the revenue from Google Cloud for 2023 and 2024 ?"]

batch_queries = processor.process_queries(query).to(model.device)
with torch.no_grad():
  query_embeddings = model(**batch_queries)

### 4. Setup FAISS vector DB Index

In [ ]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

In [ ]:
index = faiss.IndexFlatL2(128)

vector_store = FAISS(
    embedding_function=None,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

### 5. Add Embeddings & Metadata Fields to Index

In [ ]:
# import PyPDF2
# from PIL import Image

file_name = '/content/google-alphabet-2024.pdf'

all_images = []
for i, img in enumerate(images):
  page_name = "/content/"+str(i)+".pdf"
  all_images.append(page_name)

all_embeddings = [l for l in dataset]
list_of_tuple = []
for img, embd in zip(all_images, all_embeddings):
  for i in range(0,1030):
    list_of_tuple.append((img, embd[i]))

metadata = []
uids = []
count=0
for i in range(0,len(images)):
  for j in range(0,1030):
    file_name = file_name
    page_name = file_name.split("/")[-1].split(".")[0]+"_"+str(i)+".pdf"
    patch_num = j
    uid = count
    metadata.append({"file_name":file_name, "page_name": page_name, "patch_num":patch_num,"uid":uid})
    uids.append(uid)
    count += 1

In [ ]:
ids = vector_store.add_embeddings(text_embeddings= list_of_tuple, metadatas=metadata, ids = uids)

### 6. Retrieve best Matching Page from Vector DB

In [ ]:
page_name_list = []
for i in range(0,query_embeddings[0].shape[0]):
    vec = query_embeddings[0][i].tolist()
    results = vector_store.similarity_search_by_vector(vec, k=3)
    for doc in results:
       page_name_list.append(doc.metadata['page_name'])

In [ ]:
unique_list_items = set(page_name_list)
page_dict={}

for list_item in unique_list_items:
    page_dict[list_item]=page_name_list.count(list_item)
sorted_page_dict = dict(sorted(page_dict.items(),key=lambda item: item[1], reverse=True))
top_pages = list(sorted_page_dict.keys())
top_pages

### 7. Rerank Vector DB output using Colpali as Re-ranker
*  Get page embeddings from DB
*  Call score function using colpali embeddings

In [ ]:
def get_doc_embeddings(page_names_list):
  doc_vectors = []
  for page_name in page_names_list:

      page_num = page_name.split("_")[-1].split(".")[0]
      page_num = int(page_num)
      doc_vectors.append(vector_store.index.reconstruct_n(page_num*1030,1030))

  doc_embd = torch.from_numpy(np.stack(doc_vectors))
  return doc_embd

In [ ]:
doc_embd = get_doc_embeddings(top_pages)
print(doc_embd.shape)

In [ ]:
def score(query_embedding, dataset):
  scores = processor.score_multi_vector(query_embedding, dataset)
  scores = np.array(scores)
  matched_pages = scores.flatten().argsort()[::-1]
  return scores, matched_pages

scores, matched_pages = score(list(torch.unbind(query_embeddings.to("cpu").to(torch.float32))),doc_embd)
print('Similarity scores: ', scores[0], '\n', 'Matched pages: ', [top_pages[matched_pages[i]] for i in matched_pages])

In [ ]:
top_pages[matched_pages[0]]

In [ ]:
fig, ax = plt.subplots(figsize=(16, 12))
ax.imshow(images[int(top_pages[matched_pages[0]].split("_")[-1].split(".")[0])])
ax.axis("off")
plt.show()

### 8. Load Multi-modal LLM: Qwen2.5-VL 3B

In [ ]:
!pip install qwen-vl-utils==0.0.08

In [ ]:
from qwen_vl_utils import process_vision_info
from transformers import Qwen2_5_VLForConditionalGeneration, AutoTokenizer, AutoProcessor

In [ ]:
# default: Load the model on the available device(s)
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2.5-VL-3B-Instruct", torch_dtype="auto", device_map="auto"
)


# default processer
processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-3B-Instruct")

### 9. Prepare Prompt with Retrieved Page

In [ ]:
messages = [

    {"role": "user",
     "content": [
         {"type": "image",
          "image": images[int(top_pages[matched_pages[0]].split("_")[-1].split(".")[0])],
          "resized_height": 1024,
          "resized_width": 1024,
         },
        {"type": "text", "text": "What is the revenue from Google Cloud for 2023 and 2024 ?"}]},

]

In [ ]:
# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

### 10. Generate Answer: Model Inference

In [ ]:
# Inference: Generation of the output

with torch.no_grad():
  generated_ids = model.generate(**inputs, max_new_tokens=64)

generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text[0])

**Key Takeways**

* Using ColPali (VLM) as re-ranker is the best practical way to achieve best performance while scaling up your system.

**Current State of Vector DBs**

1.   Qdrant vector DB has an offering with ColPali, ColQwen integration (Jan, 2025)
2.   Weaviate DB started offering multi-vector embedding support from v1.29 (Feb, 2025)
3. Elasticsearch has rolled out ColPali implementation in March, 2025.
4. Milvus vector DB completed integration with ColPali (multi-vector) by May, 2025.
